## Thông tin nhóm

**Thành viên thứ 1:**

- Họ và tên: Trần Trung Hiếu
- MSSV: 1712442

**Thành viên thứ 2:**
- Họ và tên: Lê Long Hồ
- MSSV: 1712447

**Thành viên thứ 3:**
- Họ và tên: Nguyễn Đình Thiên Phúc
- MSSV: 18120144

___

## Import

In [2]:
!pip3 install nltk
!pip3 install pyvi

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import re
from pyvi import ViTokenizer
from nltk.stem.porter import PorterStemmer
import string

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

pd.set_option('display.max_rows', 500)

  Using cached https://files.pythonhosted.org/packages/34/5b/bd0f0fb5564183884d8e35b81d06d7ec06a20d1a0c8b4c407f1554691dce/joblib-1.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5a/75/aca08032c9752a75acc60ff7f4e58f1a74164b996395f44727ffdb17ebaf/regex-2020.11.13-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/80/02/8f8880a4fd6625461833abcf679d4c12a44c76f9925f92bf212bb6cefaad/tqdm-4.56.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/10/e1/0e5bc6b5e3327b9385d6e0f1b0a7c0404f28b74eb6db59a778515b30fd9c/pyvi-0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/25/74/5b7befa513482e6dee1f3dd68171a6c9dfc14c0eaa00f885ffeba54fe9b0/sklearn_crfsuite-0.3.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/pa

[nltk_data] Downloading package punkt to
[nltk_data]     /home/tthieu22_06/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---

## Thu thập dữ liệu

TODO

---

## Khám phá dữ liệu

In [3]:
movie_df = pd.read_csv('movie_final.csv')
movie_df.sample(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
4786,https://www.themoviedb.org/movie/334543,Tìm Mẹ\n(2016),81.0,poster_final/334543.jpg,PG-13,Như rất nhiều đứa trẻ sinh ra ở vùng nông thôn...,Cuộc tìm kiếm bắt đầu.,Phim Chính Kịch,NaN,vi
3410,https://www.themoviedb.org/movie/150689,Lọ Lem\n(2015),68.0,poster_final/150689.jpg,PG,Lọ Lem là một chuyện kể dân gian châu Âu ra đờ...,Midnight is just the beginning.,"Phim Lãng Mạn, Phim Giả Tượng, Phim Gia Đình, ...",NaN,vi
109,https://www.themoviedb.org/tv/14743,El Cor de la Ciutat\n(2000),70.0,NaN,NaN,El cor de la ciutat is a TVC television soap o...,NaN,Phim Chính Kịch,NaN,en
3092,https://www.themoviedb.org/movie/461083,Fate/Kaleid Liner Prisma☆Illya Movie: Oath Und...,66.0,poster_final/461083.jpg,NaN,We don't have an overview translated in Englis...,NaN,"Phim Hoạt Hình, Phim Hành Động, Phim Giả Tượng",NaN,en
3117,https://www.themoviedb.org/movie/10226,High Tension\n(2003),67.0,poster_final/10226.jpg,R,Best friends Marie and Alexia decide to spend ...,Hearts will bleed.,"Phim Kinh Dị, Phim Gây Cấn, Phim Bí Ẩn",NaN,en


In [4]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              5598 non-null   object 
 1   title            5594 non-null   object 
 2   score            5594 non-null   float64
 3   posterImagePath  5326 non-null   object 
 4   certification    4177 non-null   object 
 5   overview         5594 non-null   object 
 6   tagline          2522 non-null   object 
 7   genres           5230 non-null   object 
 8   trailerUrl       79 non-null     object 
 9   language         5594 non-null   object 
dtypes: float64(1), object(9)
memory usage: 437.5+ KB


In [5]:
n_row, n_col = movie_df.shape
print("Dữ liệu có số dòng:", n_row)
print("Dữ liệu có số cột:", n_col)

Dữ liệu có số dòng: 5598
Dữ liệu có số cột: 10


Mỗi dòng trong tập dữ liệu thể hiện vài thông tin của một bộ phim, bộ phim đó có thể là phim truyền hình hoặc phim điện ảnh. Các dòng này đều có ý nghĩa tương đương nhau. 

## Đưa ra câu hỏi cần trả lời

Các thông tin được tải lên trên Internet hầu như không được kiểm duyệt, chính vì vậy sẽ có thể xuất hiện những nội dung không phù hợp với mọi lứa tuổi, cụ thể là những bộ phim có gán nhãn giới hạn độ tuổi người lớn không nên để những người chưa đủ tuổi, đặc biệt là trẻ nhỏ tiếp xúc. Tuy nhiên, rất nhiều các website xem phim lại không có gán nhãn giới hạn độ tuổi. Chính vì vậy, câu hỏi này được đưa ra nhằm tìm câu trả lời giải quyết vấn đề nội dung bộ phim không phù hợp với các lứa tuổi, giúp các phụ huynh có thể quản lí được trẻ nhỏ không tìm coi những bộ phim có nội dung không phù hợp.

Câu hỏi này sẽ liên quan tới việc phân loại nội dung tóm tắt của một bộ phim và dự đoán độ tuổi phù hợp để coi bộ phim này là gì.

- Input: overview (nội dung tóm tắt của bộ phim)
- Output: certification (độ tuổi phù hợp)

---

## Khám phá dữ liệu

In [6]:
print("Danh sách các ngôn ngữ thể hiện nội dung tóm tắt:")
print(movie_df.language.value_counts())

Danh sách các ngôn ngữ thể hiện nội dung tóm tắt:
en    4304
vi    1224
lt      31
fi      10
tl       5
pt       4
cy       4
ro       3
es       2
fr       1
it       1
id       1
cs       1
de       1
sk       1
af       1
Name: language, dtype: int64


In [7]:
print('Kiểu dữ liệu của cột cerfitication:', movie_df.certification.dtype)

Kiểu dữ liệu của cột cerfitication: object


In [8]:
print("Số lượng dòng bị thiếu output:", movie_df.certification.isnull().sum())
print("Số lượng dòng có output:", movie_df.shape[0] - movie_df.certification.isnull().sum())

Số lượng dòng bị thiếu output: 1421
Số lượng dòng có output: 4177


In [9]:
print('Số lượng lớp trong certification:', len(movie_df.certification.unique()))
print('Tỉ lệ bộ phim của từng lớp:\n', movie_df.certification.value_counts(normalize=True)*100)

Số lượng lớp trong certification: 88
Tỉ lệ bộ phim của từng lớp:
 R             14.891070
TV-14         12.401245
PG-13         10.892985
TV-MA          9.743835
PG             9.743835
TV-PG          4.907829
G              4.141729
12             4.045966
NR             3.998085
15             3.303807
16             2.657410
TV-Y7          2.250419
TV-G           1.723725
U              1.484319
18             1.173091
14             1.125209
L              1.053388
16+            0.694278
10             0.598516
0              0.598516
6              0.550634
18+            0.478813
T              0.430931
12+            0.406991
MA15+          0.383050
TV-Y           0.359109
保護級            0.359109
AL             0.335169
普遍級            0.335169
6+             0.287288
7              0.287288
輔12級           0.239406
M              0.215466
APTA           0.215466
NC-17          0.215466
UA             0.215466
11             0.191525
M/12           0.167584
NC16           0.14364

## Tiền xử lí - tách các tập dữ liệu

### Lọc thông tin cần thiết

`en` và `vi` là hai ngôn ngữ có số lượng bộ phim chiếm nhiều nhất, cho nên nhóm sẽ giữ lại những bộ phim thuộc một trong hai ngôn ngữ này để thực hiện tiền xử lí và loại bỏ đi các dòng chứa các ngôn ngữ còn lại

In [10]:
envi_movie_df = movie_df[((movie_df.language == 'vi') | (movie_df.language == 'en'))]
envi_movie_df.reset_index(inplace=True, drop=True)
envi_movie_df.head(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
0,https://www.themoviedb.org/tv/44217,Huyền Thoại Vikings\n(2013),79.0,poster_final/44217.jpg,14+,Người Viking đưa chúng ta đến thế giới tàn khố...,NaN,"Action & Adventure, Phim Chính Kịch",NaN,vi
1,https://www.themoviedb.org/tv/82856,The Mandalorian\n(2019),85.0,poster_final/82856.jpg,TV-14,"After the fall of the Galactic Empire, lawless...",Bounty hunting is a complicated profession.,"Sci-Fi & Fantasy, Action & Adventure",NaN,en
2,https://www.themoviedb.org/tv/77169,Cobra Kai\n(2018),81.0,poster_final/77169.jpg,TV-14,This Karate Kid sequel series picks up 30 year...,Cobra Kai never dies.,"Action & Adventure, Phim Chính Kịch",NaN,en
3,https://www.themoviedb.org/tv/75006,Học Viện Umbrella\n(2019),87.0,poster_final/75006.jpg,TV-MA,"Được đưa về quá khứ những năm 1960 tại Dallas,...",Super. Dysfunctional. Family.,"Action & Adventure, Sci-Fi & Fantasy, Phim Chí...",NaN,vi
4,https://www.themoviedb.org/tv/71712,Bác Sĩ Thiên Tài\n(2017),86.0,poster_final/71712.jpg,TV-14,Một bác sĩ trẻ mắc hội chứng bác học được nhận...,"His mind is a mystery, his methods are a miracle.",Phim Chính Kịch,NaN,vi


Xóa các dòng bị thiếu nhãn `certification`

In [11]:
notnull_envi_movie_df = envi_movie_df.dropna(subset=['certification']).reset_index(drop=True)
print('Số lượng bộ phim sau khi loại bỏ nhãn thiếu:', notnull_envi_movie_df.shape[0])

Số lượng bộ phim sau khi loại bỏ nhãn thiếu: 4171


Vì số lượng lớp lớn và phân bố không đồng đều, thực hiện chia các lớp giới hạn tuổi thành 2 cụm là: lớn hơn 18 (>=18) và nhỏ hơn 18 (<18)

Danh sách cụm này được lưu trong tệp `age_restricted.txt`

In [12]:
# Danh sách các nhóm tuổi
age_cert = {}
with open('age_restricted.txt', 'r') as cert_file:
    for line in cert_file:
        cert, age = line.strip().split(',')
        age_cert[cert] = 1 if age == '>18' else 0

notnull_envi_movie_df['is_adult'] = notnull_envi_movie_df.certification.apply(lambda c: age_cert[c])

In [13]:
notnull_envi_movie_df.sample(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language,is_adult
1734,https://www.themoviedb.org/tv/2260,H2O: Just Add Water\n(2006),76.0,poster_final/2260.jpg,PG,H2O: Just Add Water revolves around three teen...,NaN,"Phim Chính Kịch, Phim Gia Đình, Sci-Fi & Fantasy",NaN,en,0
2682,https://www.themoviedb.org/movie/75656,Phi Vụ Thế Kỷ\n(2013),74.0,poster_final/75656.jpg,PG-13,Phi Vụ Thế Kỷ nói về một nhóm các nhà ảo thuật...,4 amazing magicians. 3 impossible heists. 1 bi...,"Phim Gây Cấn, Phim Hình Sự",NaN,vi,0
928,https://www.themoviedb.org/tv/648,The Incredible Hulk\n(1977),71.0,poster_final/648.jpg,TV-PG,"During an experiment gone bad, radiation turns...",NaN,"Action & Adventure, Phim Chính Kịch",NaN,en,0
4091,https://www.themoviedb.org/movie/14257,Delusions of Grandeur\n(1971),75.0,poster_final/14257.jpg,PG,Don Sallust (Louis de Funès) is minister of th...,NaN,"Phim Hài, Phim Lịch Sử",NaN,en,0
1717,https://www.themoviedb.org/tv/31295,Misfits\n(2009),76.0,poster_final/31295.jpg,18,When five young outsiders on Community Service...,NaN,"Sci-Fi & Fantasy, Phim Chính Kịch, Phim Hài",NaN,en,1


### Tách các tập dữ liệu

Vì hai ngôn ngữ `en` và `vi` có đặc điểm khác nhau, để tránh chiều dữ liệu quá lớn và làm giảm hiệu quả của mô hình phân loại, ta sẽ chia `en` và `vi` ra làm 2 tập dữ liệu riêng biệt. Đồng thời, do chỉ tập trung vào `overview` cho nên sẽ loại bỏ các cột khác chỉ giữ lại `overview` và `is_adult`

In [14]:
en_movie_df = notnull_envi_movie_df[notnull_envi_movie_df.language == 'en'][['overview', 'is_adult']]
vi_movie_df = notnull_envi_movie_df[notnull_envi_movie_df.language == 'vi'][['overview', 'is_adult']]

len(en_movie_df), len(vi_movie_df)

(2968, 1203)

Thực hiện chia thành các tập train, valid, và test với tỉ lệ lần lượt là 70/15/15

In [15]:
# Tách tập cho tiếng Việt
vi_y_sr = vi_movie_df.is_adult
vi_X_df = vi_movie_df.drop(columns=['is_adult'])

vi_train_X, vi_tmp_X, vi_train_y, vi_tmp_y = train_test_split(vi_X_df, vi_y_sr, stratify=vi_y_sr, random_state=42, test_size=0.3)
vi_valid_X, vi_test_X, vi_valid_y, vi_test_y = train_test_split(vi_tmp_X, vi_tmp_y, stratify=vi_tmp_y, random_state=42, test_size=0.15)


In [16]:
vi_train_X.shape, vi_train_y.shape

((842, 1), (842,))

In [17]:
vi_valid_X.shape, vi_valid_y.shape

((306, 1), (306,))

In [18]:
vi_test_X.shape, vi_test_y.shape

((55, 1), (55,))

In [19]:
# Tách tập cho tiếng Anh
en_y_sr = en_movie_df.is_adult
en_X_df = en_movie_df.drop(columns=['is_adult'])

en_train_X, en_tmp_X, en_train_y, en_tmp_y = train_test_split(en_X_df, en_y_sr, stratify=en_y_sr, random_state=42, test_size=0.3)
en_valid_X, en_test_X, en_valid_y, en_test_y = train_test_split(en_tmp_X, en_tmp_y, stratify=en_tmp_y, random_state=42, test_size=0.5)

In [20]:
en_train_X.shape, en_train_y.shape

((2077, 1), (2077,))

In [21]:
en_valid_X.shape, en_valid_y.shape

((445, 1), (445,))

In [22]:
en_test_X.shape, en_test_y.shape

((446, 1), (446,))

---

## Khám phá dữ liệu - tập huấn luyện

In [23]:
en_overview_dup = en_train_X.duplicated(['overview'])
print("Số lượng nội dung tóm tắt bị trùng:", en_overview_dup.sum())

print("3 nội dung tóm tắt trùng nhiều nhất:")
print(en_train_X[en_overview_dup].overview.value_counts()[:3])
    

Số lượng nội dung tóm tắt bị trùng: 292
3 nội dung tóm tắt trùng nhiều nhất:
We don't have an overview translated in English. Help us expand our database by adding one.                                                                                                                                                                                                                                                                                                                                                                                73
An edge-of-your-seat view into the lives of everyday heroes committed to one of America's noblest professions. For the firefighters, rescue squad and paramedics of Chicago Firehouse 51, no occupation is more stressful or dangerous, yet so rewarding and exhilarating. These courageous men and women are among the elite who forge headfirst into danger when everyone else is running the other way and whose actions make the difference between life and death.     2

In [26]:
word_dict = Counter()
max_leng = -1
for words in en_train_X.overview.apply(lambda s: word_tokenize(s)):
    word_dict.update(words)
    max_leng = max(max_leng, len(words))

max_leng, word_dict.most_common(20)

(302,
 [(',', 6132),
  ('the', 5523),
  ('.', 4675),
  ('a', 3547),
  ('and', 3422),
  ('of', 3141),
  ('to', 2953),
  ('in', 1876),
  ('his', 1342),
  ('is', 1326),
  ('with', 907),
  ('her', 790),
  ('on', 785),
  ("'s", 782),
  ('an', 727),
  ('by', 721),
  ('their', 690),
  ('he', 674),
  ('for', 668),
  ('as', 666)])

In [27]:
vi_overview_dup = vi_train_X.duplicated(['overview'])
print("Số lượng nội dung tóm tắt bị trùng:", vi_overview_dup.sum())

print("3 nội dung tóm tắt trùng nhiều nhất:")
print(vi_train_X[vi_overview_dup].overview.value_counts()[:3])
    

Số lượng nội dung tóm tắt bị trùng: 120
3 nội dung tóm tắt trùng nhiều nhất:
Phim Đảo Hải Tặc - One Piece là chuyện về cậu bé Monkey D. Luffy do ăn nhầm Trái Ác Quỷ, bị biến thành người cao su và sẽ không bao giờ biết bơi. 10 năm sau sự việc đó, cậu rời quê mình và kiếm đủ 10 thành viên để thành một băng hải tặc, biệt hiệu Hải tặc Mũ Rơm.Khi đó của phiêu lưu tìm kiếm kho báu One Piece bắt đầu. Trong cuộc phiêu lưu tìm kiếm One Piece, băng Hải tặc mũ rơm phải chiến đấu với nhiều băng hải tặc xấu khác cũng muốn độc chiếm One Piece và Hải quân của Chính phủ muốn diệt trừ hải tặc. Băng Hải tặc Mũ Rơm phải trải qua biết bao nhiêu khó khăn, không lùi bước với ước mơ "Trở thành Vua Hải Tặc và chiếm được kho báu One Piece".    2
Ở thế giới nơi đủ loại động vật cùng tồn tại, chú sói hiền lành nhận thức được bản năng săn mồi của mình khi ngôi trường của chú ứng phó với một kẻ sát nhân đang ẩn náu.                                                                                                    

In [36]:
word_dict = Counter()
max_leng = -1
for words in vi_train_X.overview.apply(lambda s: ViTokenizer.tokenize(s).split()):
    word_dict.update(words)
    max_leng = max(max_leng, len(words))

max_leng, word_dict.most_common(20)

(203,
 [(',', 3167),
  ('.', 2782),
  ('của', 1774),
  ('một', 1575),
  ('và', 1463),
  ('là', 969),
  ('người', 794),
  ('những', 792),
  ('trong', 664),
  ('được', 656),
  ('với', 643),
  ('đã', 607),
  ('có', 501),
  ('khi', 478),
  ('về', 452),
  ('để', 450),
  ('phim', 447),
  (')', 422),
  ('(', 421),
  ('bị', 396)])

## Tiền xử lí

In [73]:
class FeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stemmer = PorterStemmer()
        with open('en_stopwords.txt', 'r') as en_file: 
            self.en_stopwords = [line.strip() for line in en_file]
        with open('vi_stopwords.txt', 'r') as vi_file: 
            self.vi_stopwords = [line.strip() for line in vi_file]
    
    def fit(self, X, y=None, lang='en'):
        pass
    
    def transform(self, X, y=None, lang='en'):
        transformed_X = X.apply(lambda s: re.sub(f'[{string.punctuation}]', ' ', s.lower()))
        if lang == 'en':
            most_duplicate = "We don't have an overview translated in English. Help us expand our database by adding one."
            most_duplicate = re.sub(f'[{string.punctuation}]', ' ', most_duplicate.lower())
            transformed_X = transformed_X[transformed_X != most_duplicate] \
                                            .apply(lambda s: [self.stemmer.stem(word) for word in word_tokenize(s) 
                                                           if word not in self.en_stopwords])
        elif lang == 'vi':
            transformed_X = transformed_X.apply(lambda s: [word for word in ViTokenizer.tokenize(s).split() 
                                                           if word not in self.en_stopwords])
        else:
            raise Exception('we only support `en` and `vi` language')
        
        return transformed_X

In [75]:
transformer = FeatureTransformer()
transformed_en_train_X = transformer.transform(en_train_X.overview)
transformed_vi_train_X = transformer.transform(vi_train_X.overview)

## Tiền xử lí - tập validation

In [77]:
transformed_en_valid_X = transformer.transform(en_valid_X.overview)
transformed_vi_valid_X = transformer.transform(vi_valid_X.overview)

---

## Mô hình hóa dữ liệu

Huấn luyện mô hình

### Hyperparameter tuning

TODO

### Đánh giá mô hình

TODO

---

## Kết luận

TODO